In [1]:
import pymc3
import theano.tensor as tt

In [4]:
KB = 0.0019872041      # in kcal/mol/K

In [2]:
def heats_TwoComponentBindingModel(V0, DeltaVn, P0, Ls, DeltaG, DeltaH, DeltaH_0, beta, N):
    """
    Expected heats of injection for two-component binding model.

    ARGUMENTS
    V0 - cell volume (liter)
    DeltaVn - injection volumes (liter)
    P0 - Cell concentration (millimolar)
    Ls - Syringe concentration (millimolar)
    DeltaG - free energy of binding (kcal/mol)
    DeltaH - enthalpy of binding (kcal/mol)
    DeltaH_0 - heat of injection (cal)
    beta - inverse temperature * gas constant (mole / kcal)
    N - number of injections

    Returns
    -------
    expected injection heats (calorie)

    """
    Kd = tt.exp(beta * DeltaG)   # dissociation constant (M)

    # Compute complex concentrations.
    # Pn[n] is the protein concentration in sample cell after n injections
    # (M)
    Pn = tt.zeros([N])
    # Ln[n] is the ligand concentration in sample cell after n injections
    # (M)
    Ln = tt.zeros([N])
    # PLn[n] is the complex concentration in sample cell after n injections
    # (M)
    PLn = tt.zeros([N])

    dcum = 1.0  # cumulative dilution factor (dimensionless)
    for n in range(N):
        # Instantaneous injection model (perfusion)
        # dilution factor for this injection (dimensionless)
        d = 1.0 - (DeltaVn[n] / V0)
        dcum *= d  # cumulative dilution factor
        # total quantity of protein in sample cell after n injections (mol)
        P = V0 * P0 * 1.e-3 * dcum
        # total quantity of ligand in sample cell after n injections (mol)
        L = V0 * Ls * 1.e-3 * (1. - dcum)
        
        # complex concentration (M)
        PLn = tt.set_subtensor(PLn[n], (0.5 / V0 * ((P + L + Kd * V0) - tt.sqrt((P + L + Kd * V0) ** 2 - 4 * P * L) )))

        # free protein concentration in sample cell after n injections (M)
        Pn = tt.set_subtensor(Pn[n], P / V0 - PLn[n])

        # free ligand concentration in sample cell after n injections (M)
        Ln = tt.set_subtensor(Ln[n], L / V0 - PLn[n])

    # Compute expected injection heats.
    # q_n_model[n] is the expected heat from injection n
    q_n = tt.zeros([N])
    # Instantaneous injection model (perfusion)
    # first injection
    q_n = tt.set_subtensor(q_n[0], (DeltaH * V0 * PLn[0])*1000. + DeltaH_0)

    for n in range(1, N):
        d = 1.0 - (DeltaVn[n] / V0)  # dilution factor (dimensionless)
        # subsequent injections
        q_n = tt.set_subtensor(q_n[n], (DeltaH * V0 * (PLn[n] - d * PLn[n - 1])) * 1000. + DeltaH_0)

    return q_n

In [3]:
def make_TwoComponentBindingModel(q_actual_cal, exper_design_params,
                                  dcell=0.1, dsyringe=0.1,
                                  uniform_P0=False, P0_min=None, P0_max=None, 
                                  uniform_Ls=False, Ls_min=None, Ls_max=None):
    """
    to create a pymc3 model
    :param q_actual_cal: observed heats in calorie
    :param exper_design_params: dict
    :param dcell: float, relative uncertainty in cell concentration
    :param dsyringe: float, relative uncertainty in syringe concentration
    :param uniform_P0: bool
    :param uniform_Ls: bool
    :param concentration_range_factor: float, if uniform_P0, or uniform_Ls or both is True,
                                        lower = stated_value / concentration_range_factor,
                                        upper = stated_value * concentration_range_factor
    :param auto_transform: bool, to turn automatic transform on or off
    :return: an instance of pymc3.model.Model
    """

    V0 = exper_design_params["cell_volume_liter"]

    DeltaVn = exper_design_params["injection_volumes_liter"]

    beta = 1 / KB / exper_design_params["target_temperature_kelvin"]
    n_injections = exper_design_params["number_injections"]

    DeltaH_0_min, DeltaH_0_max = deltaH0_guesses(q_actual_cal)
    log_sigma_min, log_sigma_max = logsigma_guesses(q_actual_cal)

    stated_P0 = exper_design_params["cell_concentration_milli_molar"]
    print("stated_P0", stated_P0)
    uncertainty_P0 = dcell * stated_P0
    print("uncertainty_P0", uncertainty_P0)
    P0_min = stated_P0 / concentration_range_factor
    P0_max = stated_P0 * concentration_range_factor
    print("P0 [%0.5e, %0.5e]" % (P0_min, P0_max))

    stated_Ls = exper_info.get_syringe_concentration_milli_molar()
    print("stated_Ls", stated_Ls)
    uncertainty_Ls = dsyringe * stated_Ls
    print("uncertainty_Ls", uncertainty_Ls)
    Ls_min = stated_Ls / concentration_range_factor
    Ls_max = stated_Ls * concentration_range_factor
    print("Ls [%0.5e, %0.5e]" % (Ls_min, Ls_max))

    with pymc3.Model() as model:

        # prior for receptor concentration
        if uniform_P0:
            print("Uniform prior for P0")
            P0 = uniform_prior("P0", lower=P0_min, upper=P0_max, auto_transform=auto_transform)
        else:
            print("LogNormal prior for P0")
            P0 = lognormal_prior("P0", stated_value=stated_P0, uncertainty=uncertainty_P0,
                                 auto_transform=auto_transform)

        # prior for ligand concentration
        if uniform_Ls:
            print("Uniform prior for Ls")
            Ls = uniform_prior("Ls", lower=Ls_min, upper=Ls_max, auto_transform=auto_transform)
        else:
            print("LogNormal prior for Ls")
            Ls = lognormal_prior("Ls", stated_value=stated_Ls, uncertainty=uncertainty_Ls,
                                 auto_transform=auto_transform)

        # prior for DeltaG
        DeltaG = uniform_prior("DeltaG", lower=-40., upper=40., auto_transform=auto_transform)

        # prior for DeltaH
        DeltaH = uniform_prior("DeltaH", lower=-100., upper=100., auto_transform=auto_transform)

        # prior for DeltaH_0
        DeltaH_0 = uniform_prior("DeltaH_0", lower=DeltaH_0_min, upper=DeltaH_0_max, auto_transform=auto_transform)

        # prior for log_sigma
        log_sigma = uniform_prior("log_sigma", lower=log_sigma_min, upper=log_sigma_max, auto_transform=auto_transform)

        q_model_cal = heats_TwoComponentBindingModel(V0, DeltaVn, P0, Ls, DeltaG, DeltaH, DeltaH_0, beta, n_injections)

        sigma_cal = np.exp(log_sigma)

        q_model_micro_cal = q_model_cal * 10.**6
        q_actual_micro_cal = q_actual_cal * 10.**6
        sigma_micro_cal = sigma_cal * 10.**6

        q_obs = pymc3.Normal("q_obs", mu=q_model_micro_cal, sd=sigma_micro_cal, observed=q_actual_micro_cal)

    return model
